In [23]:
import mne
mne.set_log_level('ERROR')  # выводить только ошибки
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
import numpy as np
from braindecode.models import EEGNetv4 as EEGNet
import os
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import torch.optim as optim
from tqdm import tqdm

### Preparation

In [24]:
def get_filtered_data(raw):
    
    raw_resampled = raw.copy().resample(256)
    
    # Убираем дрейф и ВЧ-шум → 0.5–45 Гц, мышечные артефакты (EMG), движение кожи
    raw_filtered = raw_resampled.copy().filter(l_freq=0.5, h_freq=45)
    
    # Удаление сетевой наводки (50 Гц)
    raw_filtered.notch_filter(freqs=[50])
    
    # Референцированиe
    # Пересчёт в среднее по всем каналам
    raw_filtered.set_eeg_reference('average')
    # дополнительно можно ICA -> find_bads_eog
    
    # Нормализация
    data = raw_filtered.get_data()  # [n_channels, n_times]
    # Z-score по каналам -убрать индивидуальные различия каналов по масштабу, сделать данные сопоставимыми
    data_zscored = (data - data.mean(axis=1, keepdims=True)) / data.std(axis=1, keepdims=True)
    return data_zscored

In [25]:
# for datasets with differences in channels
# globally used in 2 classes :(
needed_channels = [
     'C3-P3',
     'C4-P4',
     'CZ-PZ',
     'F3-C3',
     'F4-C4',
     'F7-T7',
     'F8-T8',
     'FP1-F3',
     'FP1-F7',
     'FP2-F4',
     'FP2-F8',
     'FT10-T8',
     'FT9-FT10',
     'FZ-CZ',
     'P3-O1',
     'P4-O2',
     'P7-O1',
     'P7-T7',
     'P8-O2',
     'T7-FT9',
     'T7-P7',
     'T8-P8-0']
def get_valid_files(dataset):

    res_set = []

    for f, label in dataset:
        try:
            raw = mne.io.read_raw_edf(f, preload=False, verbose=False)
            channels = raw.ch_names
            if all(ch in channels for ch in needed_channels):
                res_set.append((f, label))
        except Exception as e:
            print(f"{f.name}: {e}")
            
    return res_set

In [26]:
# Get list of files and labels
from os import listdir
from os.path import isfile, join
import mne
from pathlib import Path
training_set = [("patterns/normals/" + f, [1, 0]) for f in listdir("patterns/normals/") 
                if isfile(join("patterns/normals/", f)) and not f.startswith(('chb07', 'chb09', 'chb13', 'chb15'
                                                                          ,'chb06', 'chb18', 'chb23'))]
training_seiz = [("patterns/seizures/" + f, [0, 1]) for f in listdir("patterns/seizures/") 
                if isfile(join("patterns/seizures/", f)) and not f.startswith(('chb07', 'chb09', 'chb13', 'chb15'
                                                                          ,'chb06', 'chb18', 'chb23'))]
training_set.extend(training_seiz)
print(len(training_set))


val_set = [("patterns/normals/" + f, [1, 0]) for f in listdir("patterns/normals/") 
                if isfile(join("patterns/normals/", f)) and f.startswith(('chb07', 'chb09', 'chb13', 'chb15'))]
val_seiz = [("patterns/seizures/" + f, [0, 1]) for f in listdir("patterns/seizures/") 
                if isfile(join("patterns/seizures/", f)) and f.startswith(('chb07', 'chb09', 'chb13', 'chb15'))]
val_set.extend(val_seiz)
print(len(val_set))


test_set = [("patterns/normals/" + f, [1, 0]) for f in listdir("patterns/normals/") 
                if isfile(join("patterns/normals/", f)) and f.startswith(('chb06', 'chb18', 'chb23'))]
test_seiz = [("patterns/seizures/" + f, [0, 1]) for f in listdir("patterns/seizures/") 
                if isfile(join("patterns/seizures/", f)) and f.startswith(('chb06', 'chb18', 'chb23'))]
test_set.extend(test_seiz)
print(len(test_set))


res_training_set = get_valid_files(training_set)
res_val_set = get_valid_files(val_set)
res_test_set = get_valid_files(test_set)

print(len(res_training_set), len(res_val_set), len(res_test_set))

2161
454
258
2131 342 253


#### HEAD

In [27]:
# Head
# load backbone
backbone = EEGNet(
    n_chans=22, 
    n_outputs=128, 
    n_times=512,
    final_conv_length='auto'
)
backbone.classify = False
backbone.load_state_dict(torch.load("backbone_ssl.pt"))
backbone = backbone.cuda()

# Head class for classification/regression
class Head(nn.Module):
    def __init__(self, in_dim=128, out_dim=2):  # out_dim=2 -> 2 pattern first
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, out_dim)
        )
    def forward(self, x):
        return self.fc(x)

head = Head(in_dim=128, out_dim=2).cuda()

optimizer = torch.optim.Adam(list(backbone.parameters())+list(head.parameters()), lr=1e-4)
# Для создания нескольких независимых друг от друга паттернов (мульти-лейбл) нужен .Sigmoid()
# функция ниже как то применяет этот сигмоид у себя внутри
criterion = nn.BCEWithLogitsLoss()


In [34]:
# Dataset with labels
class EEGPatternDataset(Dataset):
    def __init__(self, file_label_pairs, window_size=512, cut_step=256):
        self.samples, self.labels = [], []
        for f, y in file_label_pairs:
            raw = mne.io.read_raw_edf(f, preload=True)
            # select only chanells above (for datasets with different channels)
            raw.pick(needed_channels)
            # filter raw record, return some numpy obj
            data = get_filtered_data(raw)
            # превращает обычный список в торч тензор
            # y = torch.tensor(label, dtype=torch.float32)
            
            for start in range(0, data.shape[1]-window_size, cut_step):
                segment = data[:, start:start+window_size]
                
                self.samples.append(segment.astype(np.float32))
                # гарантируем, что y — float32 tensor
                label_tensor = torch.tensor(y, dtype=torch.float32)
                self.labels.append(label_tensor)  # метка паттерна
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        x = torch.tensor(self.samples[idx], dtype=torch.float32)
        y = self.labels[idx]
        return x, y


In [36]:
def train_head(backbone, head, train_loader, val_loader, epochs=10, lr=1e-3, fine_tune=False):
    
    
    # saving histories
    train_losses = []
    val_losses = []
    val_accuracies = []
    
    # Заморозить backbone
    for p in backbone.parameters():
        p.requires_grad = False

    optimizer = optim.Adam(head.parameters(), lr=lr)
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        head.train()
        total_loss = 0

        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            # y.float().unsqueeze(1).cuda()
            x, y = x.cuda(), y.float().cuda()
            with torch.no_grad():
                z = backbone(x)
            preds = head(z)
            loss = criterion(preds, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Train loss: {total_loss / len(train_loader):.4f}")
        avg_train_loss = total_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Валидация
        
        # Предсказания и метки собираем по всей валидации
        all_preds = []
        all_labels = []
        head.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.cuda(), y.cuda()
                z = backbone(x)
                preds = head(z)
                # метрики для анализа F1
                all_preds.append(preds.cpu())
                all_labels.append(y.cpu())
                # ошибка
                loss = criterion(preds, y)
                val_loss += loss.item()
                correct += ((preds > 0.5).float() == y).sum().item()
                total += y.size(0)
                
        
        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        val_acc = correct / total
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch+1}/{epochs} |"
              f" Train loss: {avg_train_loss:.4f} | Val loss: {avg_val_loss:.4f} | Val acc: {val_acc:.3f}")
        
        all_preds = torch.cat(all_preds).numpy()
        all_labels = torch.cat(all_labels).numpy()
        # Бинаризация (0.5 порог)
        binary_preds = (all_preds > 0.5).astype(int)

        precision = precision_score(all_labels, binary_preds)
        recall = recall_score(all_labels, binary_preds)
        f1 = f1_score(all_labels, binary_preds)

        print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1: {f1:.3f}")

        
    # ==== 4. Fine-tuning ====
    if fine_tune:
        print("\nРазмораживаем верхние слои backbone для тонкой настройки...")
        for p in list(backbone.parameters())[-6:]: # total = 12 (4-6 to unfreeze is ok)
            p.requires_grad = True

        optimizer = optim.Adam(list(backbone.parameters())[-6:] + list(head.parameters()), lr=1e-4)

        for epoch in range(3):  # несколько эпох fine-tuning
            head.train()
            for x, y in tqdm(train_loader, desc=f"Fine-tune epoch {epoch+1}"):
                # y.float().unsqueeze(1).cuda()
                x, y = x.cuda(), y.float().cuda()
                z = backbone(x)
                preds = head(z)
                loss = criterion(preds, y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            print(f"Fine-tune epoch {epoch+1} loss: {total_loss / len(train_loader):.4f}")

        print("Fine-tuning завершён!")
        

    # Сохранение весов 
    torch.save(head.state_dict(), "head.pt")
    torch.save(backbone.state_dict(), "backbone_finetuned.pt")

    # Сохранение истории метрик 
    history = pd.DataFrame({
        "train_loss": train_losses,
        "val_loss": val_losses,
        "val_accuracy": val_accuracies
    })
    history.to_csv("training_history_head.csv", index=False)

In [8]:
# TODO: 
# 5) how to use: after fine-tune validate head -> may need if-else block in code above
# 3) Load backbone for 4 epoch (as the best per val-loss) -> >:( something broke >:(
# 4) F1, recall, precision -> +, можно ещё ROC-AUC / PR-AUC

In [37]:
# res_training_set initialized earlier
# res_val_set initialized earlier
train_ds = EEGPatternDataset(res_training_set)
val_ds = EEGPatternDataset(res_val_set)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16)

# head initialized earlier
# backbone initialized earlier

train_head(backbone, head, train_loader, val_loader, epochs=10, fine_tune=False)

Epoch 1/10:   0%|                                      | 0/1567 [00:00<?, ?it/s]/pytorch/aten/src/ATen/native/cuda/Loss.cu:90: operator(): block: [0,0,0], thread: [1,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:90: operator(): block: [0,0,0], thread: [2,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:90: operator(): block: [0,0,0], thread: [5,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:90: operator(): block: [0,0,0], thread: [6,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:90: operator(): block: [0,0,0], thread: [7,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:90: operator(): block: [0,0,0], thread: [11,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/pytorch/aten/src/ATen/na

AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
